# From the Mouths of Babes

**Inspired by:** Washington, B., Radifera, S., Struhl, M. K., & Schulz, L. (2024). _From the mouths of babes: Toddlers' early word production favors information in common ground._ In Proceedings of the Annual Meeting of the Cognitive Science Society (Vol. 46).

In [1]:
from memo import memo
from enum import IntEnum

In [2]:
class Vocab(IntEnum):
    Ball = 0
    Bear = 1
    Duck = 2
    Milk = 3
    Shoe = 4

@memo
def novel_event_inform[fixed_event: Vocab, novel_event: Vocab, utterance: Vocab](β):
    baby: knows(fixed_event, novel_event)
    baby: wants(inform=Pr[parent.referent_guess == novel_event])
    baby: thinks[
        parent: thinks[
            scene: chooses(fixed_event in Vocab, wpp=1),
            scene: chooses(novel_event in Vocab, wpp=1)
        ],
        parent: observes [scene.fixed_event] is fixed_event,
        parent: thinks[
            baby: knows(scene.fixed_event, scene.novel_event),
            baby: chooses(referent in Vocab, wpp=referent == scene.fixed_event or referent == scene.novel_event),
            baby: chooses(utterance in Vocab, wpp=(utterance == referent) + 0.25)
        ]
    ]
    baby: chooses(utterance in Vocab, wpp=exp(β * EU[inform]))
    baby: thinks[
        parent: observes [baby.utterance] is utterance,
        parent: guesses(referent_guess in Vocab, wpp=Pr[referent_guess == baby.referent])
    ]
    return Pr[baby.utterance == utterance]

@memo
def novel_event_connect[fixed_event: Vocab, novel_event: Vocab, utterance: Vocab](β):
    baby: knows(fixed_event, novel_event)
    baby: wants(connect=Pr[parent.referent_guess == referent])
    baby: thinks[
        parent: thinks[
            scene: chooses(fixed_event in Vocab, wpp=1),
            scene: chooses(novel_event in Vocab, wpp=1)
        ],
        parent: observes [scene.fixed_event] is fixed_event,
        parent: thinks[
            baby: knows(scene.fixed_event, scene.novel_event),
            baby: chooses(referent in Vocab, wpp=referent == scene.fixed_event or referent == scene.novel_event),
            baby: chooses(utterance in Vocab, wpp=(utterance == referent) + 0.25)
        ]
    ]
    baby: chooses(referent in Vocab,  wpp=exp(β * EU[connect]))
    baby: chooses(utterance in Vocab, wpp=exp(β * EU[connect]))
    baby: thinks[
        parent: observes [baby.utterance] is utterance,
        parent: guesses(referent_guess in Vocab, wpp=Pr[referent_guess == baby.referent])
    ]
    return Pr[baby.utterance == utterance]

In [3]:
from ipywidgets import interact
from matplotlib import pyplot as plt

@interact
def draw(fixed_event=Vocab, novel_event=Vocab, β=2.0):
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.title('Goal: inform')
    plt.bar(Vocab, novel_event_inform(β)[fixed_event, novel_event])
    plt.xticks(Vocab, [v.name for v in Vocab])
    plt.ylim(0, 1)
    plt.xlabel('Utterance')
    plt.ylabel('P(child produces utterance)')

    plt.subplot(1, 2, 2)
    plt.title('Goal: connect')
    plt.bar(Vocab, novel_event_connect(β)[fixed_event, novel_event])
    plt.xticks(Vocab, [v.name for v in Vocab])
    plt.ylim(0, 1)
    plt.xlabel('Utterance')
    plt.ylabel('P(child produces utterance)')

    plt.suptitle('Model predictions when the patient is ambiguous')
    plt.tight_layout()

interactive(children=(Dropdown(description='fixed_event', options={'Ball': <Vocab.Ball: 0>, 'Bear': <Vocab.Bea…